In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
import keras
from keras.models import *
from keras.layers import *

Using TensorFlow backend.


In [3]:
%matplotlib inline
import keras 
import numpy as np 
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import itertools
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.layers import Dense, Flatten, Dropout
from tensorflow.python.client import device_lib
# keras library import  for Saving and loading model and weights
from keras.models import model_from_json
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from keras.models import Model
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Conv2DTranspose, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add
import tensorflow as tf

In [4]:
import os

In [5]:
original = os.listdir("Shenzen Dataset")

In [6]:
original.sort()

In [7]:
import cv2
import matplotlib.pyplot as plt

In [8]:
original[0]

'.ipynb_checkpoints'

In [9]:
def clahe(bgr):
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)

    lab_planes = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))

    lab_planes[0] = clahe.apply(lab_planes[0])

    lab = cv2.merge(lab_planes)

    bgr1 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return bgr1


In [10]:
x = np.zeros((len(original), 262, 262, 3), dtype=np.float32)
count = 0
for file in original[1:]:
    count = count + 1
    n = original.index(file)
    path1 = os.path.join('Shenzen Dataset/', file)
    img = cv2.imread(path1, 1)
    try:
        img = cv2.resize(img, (262, 262))
        img1 = clahe(img)
        x[n] = img1/255
    except Exception as  e:
        print(e)
        print(path1)
        print(count)

In [11]:
x.shape

(660, 262, 262, 3)

In [12]:
data = pd.read_csv("shenzhen_metadata.csv")

In [13]:
data

,study_id,sex,age,findings
0,CHNCXR_0001_0.png,Male,45,normal
1,CHNCXR_0002_0.png,Male,63,normal
2,CHNCXR_0003_0.png,Female,48,normal
3,CHNCXR_0004_0.png,Male,58,normal
4,CHNCXR_0005_0.png,Male,28,normal
...,...,...,...,...
657,CHNCXR_0658_1.png,Male,41,bilateral secondary PTB with right pneumothorax
658,CHNCXR_0659_1.png,Male,33,secondary PTB in the left upper field
659,CHNCXR_0660_1.png,Male,50,1.bilateral secondary PTB with right upper at...
660,CHNCXR_0661_1.png,Male,26,bilateral secondary PTB with right pleural th...


In [14]:
data.findings.value_counts()

normal                                                     326
bilateral PTB                                               61
Right PTB                                                   45
PTB in the right upper field                                27
left PTB                                                    19
                                                          ... 
bilateral secondary PTB  with right pneumothorax             1
secondary PTB  in the left lower field                       1
Right PTB with fibrous changes                               1
secondary PTB in the right upper field                       1
Bilateral secondary PTB with left lower field pneumonia      1
Name: findings, Length: 75, dtype: int64

In [15]:
#Better to turn every non"Normal" findings as TB. To make a Binary Classfier model.

In [16]:
a = data[data["findings"] != "normal"].index

In [17]:
data1 = data

In [18]:
data1.findings[a] = "TB"

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [19]:
#So, we have a pretty balanced Dataset with the same amount of norma and TB values.
data.findings.value_counts()

TB        336
normal    326
Name: findings, dtype: int64

In [20]:
data[data.study_id == "CHNCXR_0119_0.png"],data[data.study_id == "CHNCXR_0377_1.png"],data[data.study_id == "CHNCXR_0538_1.png"],

(              study_id   sex  age findings
 118  CHNCXR_0119_0.png  Male   20   normal,
               study_id   sex  age findings
 376  CHNCXR_0377_1.png  Male   25       TB,
               study_id   sex  age findings
 537  CHNCXR_0538_1.png  Male   38       TB)

In [21]:
findings = data.findings

In [22]:
data.drop([118, 376, 537],inplace = True)

In [23]:
data[data.study_id == "CHNCXR_0538_1.png"]

,study_id,sex,age,findings


In [24]:
data = data.reset_index()

In [25]:
Y = data.findings.to_numpy()

In [26]:
Y.shape

(659,)

In [27]:
Y = Y.reshape(Y.shape[0], 1)

In [28]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
Y = enc.fit_transform(Y)

In [29]:
Y.shape

(659, 2)

In [30]:
X = x[1:]

In [31]:
X.shape

(659, 262, 262, 3)

In [32]:
from sklearn.model_selection import train_test_split
X_train, x_test, Y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [33]:
X_train.shape, x_test.shape

((461, 262, 262, 3), (198, 262, 262, 3))

In [34]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.33, random_state=42)

In [35]:
x_train.shape, x_val.shape

((308, 262, 262, 3), (153, 262, 262, 3))

In [1]:
!pip install git+https:///github.com/qubvel/segmentation_models

  Cloning https:///github.com/qubvel/segmentation_models to /tmp/pip-req-build-90ofhglx
  Running command git clone -q https:///github.com/qubvel/segmentation_models /tmp/pip-req-build-90ofhglx
  Running command git submodule update --init --recursive -q
  Created wheel for segmentation-models: filename=segmentation_models-1.0.1-py3-none-any.whl size=33792 sha256=d6a8cc29a62cf76b9c16ba32c44ffd3cdf37c5681b46004ac344cab0a6230a0b
  Stored in directory: /tmp/pip-ephem-wheel-cache-7c_boqko/wheels/0c/6e/ce/531a110b340b976440ab8d88c225ba2fdef043b8724973b97b
Successfully built segmentation-models


In [37]:
model = tf.keras.models.Sequential([  
    tf.keras.layers.Conv2D(3, 3, activation='relu', input_shape=(262, 262,3),data_format="channels_last"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, 3, activation='relu',data_format="channels_last"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.applications.MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers. Flatten(),
    tf.keras.layers.Dropout(0.35),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dropout(0.35),
    tf.keras.layers.Dense(units=2, activation='softmax'),
])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 0s 0us/step


In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 260, 260, 3)       84        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 259, 259, 3)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 259, 259, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 257, 257, 3)       84        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 256, 256, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 3)       12        
_________________________________________________________________
mobilenetv2_1.00_224 (Model) (None, 8, 8, 1280)        2

In [41]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=tf.keras.optimizers.Adam(lr=1e-5))


In [46]:
history2 = model.fit(x = X_train, y = Y_train,
        epochs=5,
        batch_size = 10,
        validation_data=(x_val, y_val),
        shuffle=True)

Train on 461 samples, validate on 153 samples
Epoch 1/5
461/461 [==============================] - 104s 225ms/sample - loss: 0.0183 - acc: 0.9935 - val_loss: 0.0733 - val_acc: 0.9739
Epoch 2/5
461/461 [==============================] - 103s 224ms/sample - loss: 0.0207 - acc: 0.9957 - val_loss: 0.0668 - val_acc: 0.9739
Epoch 3/5
461/461 [==============================] - 104s 226ms/sample - loss: 0.0053 - acc: 1.0000 - val_loss: 0.0538 - val_acc: 0.9804
Epoch 4/5
461/461 [==============================] - 102s 220ms/sample - loss: 0.0076 - acc: 0.9957 - val_loss: 0.0467 - val_acc: 0.9869
Epoch 5/5
461/461 [==============================] - 104s 225ms/sample - loss: 0.0051 - acc: 1.0000 - val_loss: 0.0437 - val_acc: 0.9935


In [ ]:
model.evaluate(x_test, y_test)

In [49]:
predictions = model.predict(x_test)
y_pred = np.argmax(predictions, axis=1)

In [50]:
y_pred

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0])

In [51]:
y_test[:,1]

array([0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0.])

In [52]:
conf = confusion_matrix(y_test[:,1], y_pred)

In [53]:
import sklearn
print(sklearn.metrics.classification_report(y_test[:,1], y_pred))

              precision    recall  f1-score   support

         0.0       0.84      0.84      0.84        94
         1.0       0.86      0.86      0.86       104

    accuracy                           0.85       198
   macro avg       0.85      0.85      0.85       198
weighted avg       0.85      0.85      0.85       198

